In [7]:
import altair as alt
import pandas as pd

path = "/content/800m_femenino.csv"

try:

    df = pd.read_csv(path, sep=";", encoding="latin-1")
except FileNotFoundError:
    print(f"No se encontró el archivo en {path}. Verifica la ruta.")


def parse_time(x):
    if pd.isna(x):
        return None
    s = str(x).lower().replace(" min", "").strip()
    s = s.replace(",", ".")

    parts = s.split('.')
    try:
        if len(parts) == 3:
            minutes = int(parts[0])
            seconds = int(parts[1])
            hundredths = int(parts[2])
            return minutes * 60 + seconds + hundredths / 100.0
        elif len(parts) == 2:
            return float(s)
    except ValueError:
        return None
    return None

df["tiempo_segundos"] = df["Marca"].apply(parse_time)


df = df.dropna(subset=["tiempo_segundos", "Año"]).copy()
df = df[df["Año"] >= 2019].copy()
df["Año"] = df["Año"].astype(int)
df["Prueba"] = df["Prueba"].str.strip()


df['Categoria'] = df['Categoria'].replace({
    'Adulto Femenino': 'Adulto',
    'Juvenil/Junior Femenino': 'Junior'
})


chart = alt.Chart(df).mark_circle(size=80).encode(
    x=alt.X('Año:O', title='Año de Competencia'),
    y=alt.Y('tiempo_segundos:Q',
            title='Tiempo (Segundos) - Menor es mejor',
            scale=alt.Scale(zero=False, padding=1)),
    color=alt.Color('Categoria:N', legend=alt.Legend(title="Categoría"), scale=alt.Scale(
        domain=['Junior', 'Adulto'],
        range=['#ffc340', '#e28000']
    )),
    tooltip=[
        alt.Tooltip('Nombre Atleta', title='Atleta'),
        alt.Tooltip('Marca', title='Marca Oficial'),
        alt.Tooltip('tiempo_segundos', title='Segundos', format='.2f'),
        alt.Tooltip('Competencia'),
        alt.Tooltip('Año'),
        alt.Tooltip('Categoria')
    ]
).properties(
    title='Evolución de Marcas 800m Femenino: Junior vs Adulto (2019-2025)',
    width=600,
    height=400
).interactive()


nombre_archivo = 'comparacion_800m_femenino.html'
chart.save(nombre_archivo)

print(f"¡Listo! El gráfico corregido se ha guardado como '{nombre_archivo}'.")
chart

¡Listo! El gráfico corregido se ha guardado como 'comparacion_800m_femenino.html'.


alt.Chart(...)